In [ ]:
from ultralytics import YOLO
import os

import cv2
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import pickle
import tqdm

In [ ]:
colors = [
    [255, 0, 0],        # Красный
    [0, 150, 0],        # Зеленый
    [0, 0, 255],        # Синий
    [120, 120, 0],      # Желтый
    [255, 0, 255],      # Фиолетовый
    [0, 255, 255],      # Бирюзовый
    [128, 0, 0],        # Темно-красный
    [0, 128, 0],        # Темно-зеленый
    [0, 0, 128],        # Темно-синий
    [128, 128, 0],      # Темно-желтый
    [128, 0, 128],      # Темно-фиолетовый
    [0, 128, 128],      # Темно-бирюзовый
    [67, 67, 67],    # Серый
    [255, 165, 0],      # Оранжевый
    [67, 0, 130],    # Розовый
    [139, 69, 19],      # Коричневый
    [169, 169, 169],    # Темно-серый
    [12, 138, 14],    # Светло-зеленый
    [255, 20, 147],     # Гелиотроп
    [255, 99, 71]       # Темно-красный
]

In [ ]:
def draw_boxes(result):
    img = result.orig_img.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    boxes = result.boxes
    classes = boxes.cls.to('cpu').numpy().astype(int)
    conf = boxes.conf.to('cpu').numpy()
    boxes = list(boxes.xyxy.to('cpu').numpy().astype(int))

    labels = []
    for box,class_index,conf  in zip(boxes, classes, conf):
        labels.append(result.names[class_index])
        img = cv2.rectangle(img=img, pt1=box[:2], pt2=box[2:], color=colors[class_index], thickness=2)
        img = cv2.putText(img, result.names[class_index], (box[0],box[1] - 2), 
                          fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=colors[class_index],thickness=1)
    return img, labels
    # plt.figure(figsize=(6,6))
    # plt.imshow(img)

In [ ]:
def get_annotations(label_file):
    with open(label_file) as f:
        annotations = f.read().splitlines()
        annotations = [ x.split(' ') for x in annotations]
        for index, _ in enumerate(annotations):
            annotations[index][1:] = [float(x) for x in annotations[index][1:]]
            annotations[index][0] = int(annotations[index][0])
    return annotations

def draw_annotations(image_file, label_file, scale = 2):
    annotations = get_annotations(label_file)

    labels = []

    image = cv2.imread(image_file, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Размеры оригинального изображения
    height, width, _ = image.shape
    image = cv2.resize(image, (width*scale, height*scale), 
               interpolation = cv2.INTER_LINEAR)
    
    # Размеры ресайзнутого изображения
    height, width, _ = image.shape
    
    

    # Отобразить аннотации на изображении
    for annotation in annotations:
        class_index, x_center, y_center, w, h = annotation
        class_index = int(class_index)
        labels.append(class_names[class_index])
        
        # Преобразование относительных координат в абсолютные
        x = int(x_center * width)
        y = int(y_center * height)
        w = int(w * width)
        h = int(h * height)

        # Вычислить координаты углов прямоугольника
        x_min = int(x - w / 2)
        y_min = int(y - h / 2)
        x_max = int(x + w / 2)
        y_max = int(y + h / 2)

        # Отобразить прямоугольник на изображении
        image = cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color=colors[class_index], thickness=3)




        fontScale = 1

        image = cv2.rectangle(image, (x_min, y_min - 2), ((x_min + int(fontScale*20*len(class_names[class_index]))), y_min - int(fontScale*27)), 
                        color=colors[class_index], thickness=-1)
        # Добавить метку класса
        image = cv2.putText(image, class_names[class_index], (x_min, y_min - 2), 
                          fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=fontScale, color=[255,255,255],thickness=2)
        
    return image, labels


## Pascal VOC

In [ ]:
class_names = ['aeroplane' ,'bicycle' ,'bird' ,'boat' ,'bottle' ,
               'bus' ,'car' ,'cat' ,'chair' ,'cow' ,'diningtable' ,
               'dog' ,'horse' ,'motorbike' ,'person' ,'pottedplant' ,
               'sheep' ,'sofa' ,'train' ,'tvmonitor']



In [ ]:
dataset_dir = '../datasets/VOC'
split = 'test'
files = sorted([x.split('.')[0] for x in os.listdir(f'{dataset_dir}/images/{split}')])
save_dir = '../data/outputs/gd_vs_orig'


os.makedirs(save_dir, exist_ok=True)

###########################
for index in tqdm.tqdm(files):
    image_file = f'{dataset_dir}/images/{split}/{index}.jpg'
    label_file_orig = f'{dataset_dir}/labels/{split}_orig/{index}.txt'
    label_file_GD = f'{dataset_dir}/labels/{split}_GD/{index}.txt'


    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 15))
    ann = draw_annotations(image_file, label_file_orig)
    pred = draw_annotations(image_file, label_file_GD)
    # print('predictions - ', pred[1])
    # print('annotations - ', ann[1])

    axs[0].axis('off')


    axs[0].imshow(pred[0])
    axs[0].set_title('Grounding DINO pseudo labels',fontsize=20)


    axs[1].axis('off')
    axs[1].set_title('Annotations',fontsize=20)
    axs[1].imshow(ann[0])

    fig.savefig(f'{save_dir}/{index}.png')

In [ ]:
image_file = f'{dataset_dir}/images/{split}/{index}.jpg'
label_file_orig = f'{dataset_dir}/labels/{split}_orig/{index}.txt'
ann = draw_annotations(image_file, label_file_orig)
print('annotations - ', ann[1])
plt.imshow(ann[0])

In [ ]:
dataset_dir = '../datasets/VOC'
split = 'test'
files = sorted([x.split('.')[0] for x in os.listdir(f'{dataset_dir}/images/{split}')])
os.makedirs('images/gd_vs_orig', exist_ok=True)

###########################
index = files[9]
image_file = f'{dataset_dir}/images/{split}/{index}.jpg'
label_file_orig = f'{dataset_dir}/labels/{split}_orig/{index}.txt'
label_file_GD = f'{dataset_dir}/labels/{split}_GD/{index}.txt'


fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 15))
ann = draw_annotations(image_file, label_file_orig)
pred = draw_annotations(image_file, label_file_GD)
print('predictions - ', pred[1])
print('annotations - ', ann[1])

axs[0].axis('off')


axs[0].imshow(pred[0])
axs[0].set_title('Grounding DINO pseudo labels',fontsize=20)


axs[1].axis('off')
axs[1].set_title('Annotations',fontsize=20)
axs[1].imshow(ann[0])

# draw_annotations(image_file, label_file_orig)
# plt.show()

# filename = 
fig.savefig(f'images/gd_vs_orig/{index}.png')



In [ ]:
def draw_legend_existing(class_names, image = None, existing_labels = None):
    if image is None:
        background = np.ones((375*2, 150*2, 3), dtype=np.uint8)*255
    else:
        background = np.ones((image.shape[0] *2, 300, 3)) *255


    t_v_padding = 100
    t_h_padding = 30

    b_v_padding = 90
    b_h_padding = 10


    step = background.shape[0]//len(class_names)
    sq_size = 10


    if existing_labels is not None:
        label_indexes = [class_names.index(x) for x in existing_labels]
        class_names = existing_labels
    else:
        label_indexes = [x for x in range(len(class_names))]

    
    for index, (name, label_index) in enumerate(zip(class_names, label_indexes)):
        y = step * index
        # Отобразить прямоугольник на изображении

        y_min = y - sq_size//2 + b_v_padding
        y_max = y + sq_size//2 + b_v_padding

        image = cv2.rectangle(background, (b_h_padding, y_min), (b_h_padding + sq_size, y_max), color=colors[label_index], thickness=cv2.FILLED)
        
        # Добавить метку класса
        image = cv2.putText(image, class_names[index], (t_h_padding, y + t_v_padding), 
                          fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.7, color=[0,0,0], thickness=1)
    return image

In [ ]:
index = '000006'
image_file = f'datasets/VOC_pseudo_labels/images/test/{index}.jpg'
label_file = f'datasets/VOC_pseudo_labels/labels/test/{index}.txt'
results = model_voc(source=image_file, device=0)
plt.figure(figsize=(20,8))
plt.subplot(1,4,1)
annotated_img, annotated_labels = draw_annotations(image_file, label_file)
plt.axis('off')
plt.imshow(annotated_img)
plt.title('annotations')

plt.subplot(1,4,2)
results_voc = model_voc(source=image_file, device=0)
predict_img, predict_labels = draw_boxes(results_voc[0])
plt.axis('off')
plt.imshow(predict_img)
plt.title('predictions VOC')

plt.subplot(1,4,3)
results_pseudo_voc = model_pseudo_voc(source=image_file, device=0)
predict_img, predict_labels = draw_boxes(results_pseudo_voc[0])
plt.axis('off')
plt.imshow(predict_img)
plt.title('predictions pseudo VOC')

existing_labels = list(set(annotated_labels) | set(predict_labels))

plt.subplot(1,4,4)
plt.imshow(draw_legend_existing(class_names, image = None, existing_labels=existing_labels))
plt.axis('off')


plt.show()



In [ ]:
plt.figure(figsize=(20,8))
annotated_img, annotated_labels = draw_annotations(image_file, label_file)
plt.axis('off')
plt.imshow(annotated_img)
plt.title('annotations')

## Vehicle Counting 

In [ ]:
class_names = ['bus','car','motorcycle','truck']

In [ ]:
!ls ../datasets/Vehicle/labels/val_GD

In [ ]:
dataset_dir = '../datasets/Vehicle'
split = 'val'
files = sorted([x.split('.')[0] for x in os.listdir(f'{dataset_dir}/images/{split}')])
os.makedirs('Vehicle_outputs/images_gd_vs_orig', exist_ok=True)

###########################
index = files[0]
image_file = f'{dataset_dir}/images/{split}/{index}.jpeg'
label_file_orig = f'{dataset_dir}/labels/{split}_orig/{index}.txt'
label_file_GD = f'{dataset_dir}/labels/{split}_GD/{index}.txt'


fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 15))
ann = draw_annotations(image_file, label_file_orig)
pred = draw_annotations(image_file, label_file_GD)
print('predictions - ', pred[1])
print('annotations - ', ann[1])

axs[0].axis('off')


axs[0].imshow(pred[0])
axs[0].set_title('Grounding DINO pseudo labels',fontsize=20)


axs[1].axis('off')
axs[1].set_title('Annotations',fontsize=20)
axs[1].imshow(ann[0])

# draw_annotations(image_file, label_file_orig)
# plt.show()

# filename = 
fig.savefig(f'Vehicle_outputs/images_gd_vs_orig/{index}.png')

In [ ]:
dataset_dir = '../datasets/Vehicle'
split = 'val'
files = sorted([x.split('.')[0] for x in os.listdir(f'{dataset_dir}/images/{split}')])
save_dir = '../data/Vehicle_outputs/images_gd_vs_orig'
os.makedirs(save_dir, exist_ok=True)


os.makedirs(save_dir, exist_ok=True)

###########################
for index in tqdm.tqdm(files):
    image_file = f'{dataset_dir}/images/{split}/{index}.jpeg'
    label_file_orig = f'{dataset_dir}/labels/{split}_orig/{index}.txt'
    label_file_GD = f'{dataset_dir}/labels/{split}_GD/{index}.txt'


    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 15))
    ann = draw_annotations(image_file, label_file_orig)
    pred = draw_annotations(image_file, label_file_GD)
    # print('predictions - ', pred[1])
    # print('annotations - ', ann[1])

    axs[0].axis('off')


    axs[0].imshow(pred[0])
    axs[0].set_title('Grounding DINO pseudo labels',fontsize=20)


    axs[1].axis('off')
    axs[1].set_title('Annotations',fontsize=20)
    axs[1].imshow(ann[0])

    fig.savefig(f'{save_dir}/{index}.png')